In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
window_size = (1200, 1200)
img_name = 'blockmaze.png'

def show_mouse_loc(event, x, y, flags, param):
    clear_output(True)
    print('y coordinate: ', int(y / window_size[1] * image.shape[0]), '\n',
          'x coordinate: ', int(x / window_size[0] * image.shape[1]), sep = '')

def path_to_mouse(event, x, y, flags, param):
    global path, heatmap_copy
    y, x = int(y / window_size[1] * image.shape[0]), int(x / window_size[0] * image.shape[1])
    clear_output(True)
    print(f'y coordinate: {y}\nx coordinate: {x}')
    
    if event == cv2.EVENT_LBUTTONDOWN:
        heatmap_copy = heatmap.copy()
        path = []
        coor = dists[y, x, 1]
        while coor is not None:
            path.append(coor)
            coor = dists[(*coor, 1)]
        path.reverse()

In [ ]:
'''
    Djikstra's shortest path for standard graphs
'''

def dijkstra(graph, n):
    dists = np.tile([np.inf, None], (graph[:,:-1].max(), 1)) # 2D array. First column is distances to vertices. Second is the previous vertex needed to get to it
    dists[n, 0] = 0
    
    unvisited = list(range(len(dists))) # Unvisited vertices. algorithm ends when all are visited
    
    while unvisited:
        vertex = None
        min_dist = np.inf
        
        for i in unvisited: # Choose unvisited vertex with lowest distance
            if dists[i, 0] < min_dist:
                min_dist = dists[i, 0]
                vertex = i
       
        for unvisited_vert in unvisited:
            for i in graph: # Checks if theres a connection with the current vertex
                if (vertex == i[0] and unvisited_vert == i[1]) or (vertex == i[1] and unvisited_vert == i[0]):
                    new_dist = i[2] + dists[vertex, 0]
                    break
            else:
                continue
                
            if new_dist < dists[unvisited_vert, 0]:
                dists[unvisited_vert] = [new_dist, vertex]

        unvisited.remove(vertex)
    return dists
   
# Defining a graph. (first vertex, second vertex, distance between)
graph = np.array([
    (0,1,6),(0,4,3),(0,7,4),(0,2,2),(0,8,14),
    (1,6,3),(1,8,7),(2,3,3),(3,7,1),(3,5,3),
    (4,6,3),(4,7,2),(5,7,2),(5,6,3),(5,8,3),
    (6,7,3),(6,8,2)]) 

print(graph, '\n\n')

dists = dijkstra(graph, 0)

for idx, val in enumerate(dists):
    print(f'{idx}: {val}')window_size

In [ ]:
'''
    Dijkstra's shortest path for images e.g. a maze image
'''

def dijkstra(image, start):
    global unvisited
    count = 0
    
    height, width = image.shape

    dists = np.tile(np.array([np.inf, None]), (*image.shape[:2], 1)) # 3D array consisting of minimum distance to a node, and the previous node needed to reach it 
    dists[(*start, 0)] = 0
    
    unvisited = [(y, x) for y in range(height) for x in range(width)] # Unvisited nodes. algorithm ends when all are visited
    
    unvisited[:] = [i for i in unvisited if image[i] != 0]
    
    unvisited = set(unvisited)
    while unvisited:
        vertex = None
        min_dist = np.inf
        ### Bottleneck of the algorithm. Can do some fancy sorting stuff with more containers to fix. Chooses unvisited node with lowest distance
        for i in unvisited:
            if dists[(*i, 0)] < min_dist:
                min_dist = dists[(*i, 0)]
                vertex = i
        ###
        
        if vertex is None:
            print('Either no-where to go or starting point is unvisitable')
            break
            
        unvisited.remove(vertex)
        
        for y_adj, x_adj in ((1, 0), (0, 1), (-1, 0), (0, -1)): # ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)):    for diagonals
                if (vertex[0]+y_adj, vertex[1]+x_adj) in unvisited:
                    new_dist = dists[(*vertex, 0)] + 1 # (abs(y_adj) + abs(x_adj))**0.5    for diagonals (instead of '1')
                    if new_dist < dists[(vertex[0]+y_adj, vertex[1]+x_adj, 0)]:
                        dists[vertex[0]+y_adj, vertex[1]+x_adj] = [new_dist, vertex]
        
        image[vertex] = 55
        cv2.imshow('image', cv2.resize(image, window_size, interpolation=0))
        
        count += 1
        if count % 10 == 0:
            if cv2.waitKey(1) & 0xff == ord('q'):
                break
    
    cv2.destroyAllWindows()
    dists[dists[...,0] == np.inf] = [0, None] # Replaces unvisitable points with a distance of 0
    
    return dists

In [ ]:
image_res = (200, 200) # res for algorithm to go through
threshold = 205        # for thresholding the image into black and white. black = unvisitable

In [ ]:
'''
    Display image to see if thresholded right.
'''

orig_image = cv2.imread(img_name, 0)

image = cv2.resize(orig_image, image_res, interpolation = 0)
_, image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

cv2.imshow('original image', cv2.resize(orig_image, window_size, interpolation = 0))
cv2.imshow('thresholded image', cv2.resize(image, window_size, interpolation = 0))
cv2.setMouseCallback('thresholded image', show_mouse_loc)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
'''
    Generate distances table. Actual algorithm
'''

image = cv2.imread(img_name, 0)
image = cv2.resize(image, image_res, interpolation = 0)
_, image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

###############################
dists = dijkstra(image, (1, 0))
###############################

cv2.destroyAllWindows()

In [ ]:
'''
    Generate path lists
''' 

objectives = ((-2, -1), (1, -1), (-1, 1))    
paths = [[] for _ in range(len(objectives))]
for idx, obj in enumerate(objectives):
    obj = obj + (1,)
    coor = dists[obj]
    while coor is not None:
        paths[idx].append(coor)
        coor = dists[(*coor, 1)]
    paths[idx].reverse()
    if paths[idx] == []:
        print('Invalid coordinate:', obj[:2])

In [ ]:
'''
    Display generation of paths
'''

heatmap = cv2.applyColorMap((dists[..., 0].reshape(image.shape[:2]) / dists[..., 0].max() * 255).astype(np.uint8), cv2.COLORMAP_PARULA)
heatmap[dists[...,0] == 0] = [25, 25, 25]

cv2.imshow('image', cv2.resize(heatmap, window_size, interpolation=0))
cv2.setMouseCallback('image', show_mouse_loc)

count = 0
for idx in range(len(paths)):
    for coor in paths[idx]:
        heatmap[coor] = [0, 0, 255]
        cv2.imshow('paths', cv2.resize(heatmap, window_size, interpolation=0))
        count += 1
        if count % 5 == 0:
            if cv2.waitKey(1) & 0xff == ord('q'):
                break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
'''
    Display a path to wherever you click
'''
heatmap = cv2.applyColorMap((dists[..., 0].reshape(image.shape[:2]) / dists[..., 0].max() * 255).astype(np.uint8), cv2.COLORMAP_PARULA)
heatmap[dists[...,0] == 0] = (25, 25, 25)
heatmap_copy = heatmap.copy()

cv2.namedWindow('paths')
cv2.setMouseCallback('paths', path_to_mouse)

count = 0
path = []
while True:
    for i in path:
        heatmap_copy[i] = (0, 0, 255)
    cv2.imshow('paths', cv2.resize(heatmap_copy, window_size, interpolation=0))
    count += 1
    if count % 5 == 0:
        if cv2.waitKey(1) & 0xff == ord('q'):
            break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
'''
    Display a heatmap of shortest distances superimposed on the image
'''

heatmap = cv2.applyColorMap((dists[..., 0].reshape(image.shape[:2]) / dists[..., 0].max() * 255).astype(np.uint8), cv2.COLORMAP_WINTER)
heatmap[dists[...,0] == 0] = [50, 50, 50]
cv2.imshow('image', cv2.resize(heatmap, window_size, interpolation=0))
cv2.setMouseCallback('imag', show_mouse_loc)
cv2.waitKey(0)
cv2.destroyAllWindows()